In [1]:
file = open('/kaggle/input/lstm-next-word-prediction-data/LSTM DATA.txt', encoding='utf8')
# file = open('/kaggle/input/txt-data-for-nlp-task/LSTM.txt', encoding='utf8')

# Preprocessing Data

In [2]:
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import re
lines = file.readlines()
# lines2 = file1.readlines()

# Close the files
file.close()
# file1.close()

# Join the lines into a single string
df = ' '.join(lines)
# data1 = ' '.join(lines2)
# Combine the lines from the files into two strings
df = ' '.join(lines)
# data1 = ' '.join(lines2)

# Remove unwanted characters
df = df.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“', '').replace('”', '')
# data1 = data1.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“', '').replace('”', '')

# Concatenate the two strings into a single string
# df = data + ' ' + data1

# Additional cleaning using regex
df = re.sub(r'\ufeff', '', df)
df = re.sub(r'[“”"‘’]', '', df)

# Tokenize the combined text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([df])

# Check the vocabulary size
print(len(tokenizer.word_index))


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


7172


In [3]:
import pickle
pickle.dump(tokenizer,open('token.pkl','wb'))

In [4]:
df[:500]

'The Project Gutenberg eBook of Pride and Prejudice      This ebook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this ebook or online at www.gutenberg.org. If you are not located in the United States, you will have to check the laws of the country where you are located before using this eBook.  '

# Creating Input Sequence

In [5]:
input_sequences = []
for sentence in df.split('. '):  # Adjust the split based on your sentence structure
    tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]
    
    for i in range(1, len(tokenized_sentence)):
        input_sequences.append(tokenized_sentence[:i+1])

max_len = max([len(x) for x in input_sequences])

# Pad the sequences
padded_input_sequences = pad_sequences(input_sequences, maxlen=max_len, padding='pre')

# Separate into input (x) and output (y)
x = padded_input_sequences[:, :-1]
y = padded_input_sequences[:, -1]

print(x.shape)
print(y.shape)


(124648, 151)
(124648,)


In [6]:
max([len(x) for x in input_sequences])

152

In [87]:
input_sequences

[[1, 182],
 [1, 182, 391],
 [1, 182, 391, 1005],
 [1, 182, 391, 1005, 3],
 [1, 182, 391, 1005, 3, 299],
 [1, 182, 391, 1005, 3, 299, 4],
 [1, 182, 391, 1005, 3, 299, 4, 951],
 [1, 182, 391, 1005, 3, 299, 4, 951, 41],
 [1, 182, 391, 1005, 3, 299, 4, 951, 41, 1005],
 [1, 182, 391, 1005, 3, 299, 4, 951, 41, 1005, 23],
 [1, 182, 391, 1005, 3, 299, 4, 951, 41, 1005, 23, 21],
 [1, 182, 391, 1005, 3, 299, 4, 951, 41, 1005, 23, 21, 1],
 [1, 182, 391, 1005, 3, 299, 4, 951, 41, 1005, 23, 21, 1, 508],
 [1, 182, 391, 1005, 3, 299, 4, 951, 41, 1005, 23, 21, 1, 508, 3],
 [1, 182, 391, 1005, 3, 299, 4, 951, 41, 1005, 23, 21, 1, 508, 3, 549],
 [1, 182, 391, 1005, 3, 299, 4, 951, 41, 1005, 23, 21, 1, 508, 3, 549, 2647],
 [1,
  182,
  391,
  1005,
  3,
  299,
  4,
  951,
  41,
  1005,
  23,
  21,
  1,
  508,
  3,
  549,
  2647,
  8],
 [1,
  182,
  391,
  1005,
  3,
  299,
  4,
  951,
  41,
  1005,
  23,
  21,
  1,
  508,
  3,
  549,
  2647,
  8,
  1],
 [1,
  182,
  391,
  1005,
  3,
  299,
  4,
  951,
 

In [88]:
padded_input_sequences

array([[   0,    0,    0, ...,    0,    1,  182],
       [   0,    0,    0, ...,    1,  182,  391],
       [   0,    0,    0, ...,  182,  391, 1005],
       ...,
       [   0,    0,    0, ...,    2,  230,  126],
       [   0,    0,    0, ...,  230,  126,  524],
       [   0,    0,    0, ...,  126,  524, 1753]], dtype=int32)

In [7]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes=7173)

In [8]:
print(x.shape)
print(y.shape)

(124648, 151)
(124648, 7173)


# Model Architecture

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM,BatchNormalization
model = Sequential()
model.add(Embedding(14268, 100, input_length=151))
model.add(LSTM(128,return_sequences=True))
model.add(LSTM(128,return_sequences=True))
model.add(LSTM(128))
model.add(BatchNormalization())
model.add(Dense(7173,activation='softmax'))

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 151, 100)          1426800   
                                                                 
 lstm (LSTM)                 (None, 151, 128)          117248    
                                                                 
 lstm_1 (LSTM)               (None, 151, 128)          131584    
                                                                 
 lstm_2 (LSTM)               (None, 128)               131584    
                                                                 
 batch_normalization (Batch  (None, 128)               512       
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 7173)              925317    
                                                        

In [11]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

# Note I Have Trained This Model Before On 80 Epochs! And I Have Got Around 80% Accuracy # 

In [ ]:
model.fit(x,y,epochs=5,batch_size=64)

In [73]:
model.save('ln.h5')

# Predictor System

In [80]:
import numpy as np
import time
from googletrans import Translator
from keras.preprocessing.sequence import pad_sequences

# Create a translator object
translator = Translator()

text_bangla = "তখন তার বাবা"
text = translator.translate(text_bangla, src='bn', dest='en').text

for i in range(10):
    # Tokenize
    token_text = tokenizer.texts_to_sequences([text])[0]
    # Padding
    padded_token_text = pad_sequences([token_text], maxlen=151, padding='pre')
    # Predict
    pos = np.argmax(model.predict(padded_token_text))

    # Find the word in English
    for word, index in tokenizer.word_index.items():
        if index == pos:
            text = text + " " + word
            print("English:", text)

            # Translate the English text back to Bangla
            text_bangla = translator.translate(text, src='en', dest='bn').text
            print("Bangla:", text_bangla)
            time.sleep(2)


1/1 [==============================] - 0s 27ms/step
English: Then his father cried
Bangla: তখন তার বাবা কেঁদেছিলেন
1/1 [==============================] - 0s 29ms/step
English: Then his father cried her
Bangla: তখন তার বাবা তাকে কেঁদেছিলেন
1/1 [==============================] - 0s 38ms/step
English: Then his father cried her mother
Bangla: তখন তার বাবা তার মাকে কেঁদেছিলেন
1/1 [==============================] - 0s 27ms/step
English: Then his father cried her mother are
Bangla: তখন তার বাবা তার মা কেঁদেছিলেন
1/1 [==============================] - 0s 27ms/step
English: Then his father cried her mother are heartily
Bangla: তখন তার বাবা তার মা কেঁদেছিলেন আন্তরিকভাবে
1/1 [==============================] - 0s 27ms/step
English: Then his father cried her mother are heartily anxious
Bangla: তখন তার বাবা তার মা কেঁদেছিলেন আন্তরিকভাবে উদ্বিগ্ন
1/1 [==============================] - 0s 27ms/step
English: Then his father cried her mother are heartily anxious and
Bangla: তখন তার বাবা তার মা কেঁদেছিলে